这个章节主要讲了 视觉里程计 

1：如何 用特征点，在不同的图片中匹配。

2：三种场景的匹配方法，求解摄像机的三维运动
分别是 2d2d的匹配，2d3d的匹配，3d3d的匹配
对应的方法已经比较成熟 分别是 对级几何，pnp,icp 求解摄像机的三维运动

3：三角化法：获得二维图像上对应点的三维结构。

ch7 讲的就是 特征点法后面会讲直接法 例如光流法

特征点的提取主要是 角点检测，然后进行特征描述

角点的提取算法有很多 ：Harris角点、FAST角点、GFTT角点
SIFT、SURF、ORB

书中主要讲了orb特征点法

FAST关键点+ BRIEF描述子

特征匹配
快速近似最近邻(FLANN)
利用OpenCV的FLANN库，可以快速的进行特征匹配

书中是 ：slambook2/ch7/orb_cv.epp
项目中是feature_extraction.cpp







In [2]:
import cv2
import numpy as np
import sys

def feature_extraction(img1_path, img2_path):
    # 读取图像
    img_1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
    img_2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)

    # 初始化ORB检测器
    orb = cv2.ORB_create()

    # 检测关键点并计算描述符
    keypoints_1, descriptors_1 = orb.detectAndCompute(img_1, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(img_2, None)

    # 绘制关键点
    img_keypoints = cv2.drawKeypoints(img_1, keypoints_1, None, color=(0, 255, 0), flags=0)
    cv2.imshow("ORB特征点", img_keypoints)

    # 创建BFMatcher对象
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # 匹配描述符
    matches = bf.match(descriptors_1, descriptors_2)

    # 按距离排序
    matches = sorted(matches, key=lambda x: x.distance)

    # 计算最小距离和最大距离
    min_dist = min(matches, key=lambda x: x.distance).distance
    max_dist = max(matches, key=lambda x: x.distance).distance

    print(f"-- Max dist : {max_dist}")
    print(f"-- Min dist : {min_dist}")

    # 筛选好的匹配
    good_matches = [m for m in matches if m.distance <= max(2 * min_dist, 30.0)]

    # 绘制匹配结果
    img_matches = cv2.drawMatches(img_1, keypoints_1, img_2, keypoints_2, matches, None)
    img_good_matches = cv2.drawMatches(img_1, keypoints_1, img_2, keypoints_2, good_matches, None)

    cv2.imshow("所有匹配点对", img_matches)
    cv2.imshow("优化后匹配点对", img_good_matches)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # 等待5秒或按'q'键退出
    key = cv2.waitKey(0) & 0xFF
    if key == ord('q'):
        cv2.destroyAllWindows()
    else:
        cv2.destroyAllWindows()

    # 返回匹配结果图像，以便后续处理或保存
    return img_matches, img_good_matches

if __name__ == "__main__":
    # if len(sys.argv) != 3:
    #     print("usage: python feature_extraction.py img1 img2")
    #     sys.exit(1)
    
    # feature_extraction(sys.argv[1], sys.argv[2])
    feature_extraction("./1.png", "./2.png")


-- Max dist : 81.0
-- Min dist : 4.0


2025-02-12 09:58:22.598 Python[25704:191476] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-12 09:58:22.598 Python[25704:191476] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
cv2.destroyAllWindows()

: 

# 这个2D2D用对级几何搞
用本质矩阵的8点法来求解
对应的脚本是 pose_estimation_2d2d.cpp


In [2]:
import numpy as np
import cv2
import sys

def find_feature_matches(img1, img2):
    # 初始化
    orb = cv2.ORB_create()
    
    # 检测 Oriented FAST 角点位置
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    # 建立匹配器
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    
    # 对两幅图像中的BRIEF描述子进行匹配，使用 Hamming 距离
    matches = matcher.match(des1, des2)
    
    # 匹配点对筛选
    min_dist = min(matches, key=lambda x: x.distance).distance
    good_matches = [m for m in matches if m.distance <= max(2 * min_dist, 30.0)]
    
    return kp1, kp2, good_matches

def pose_estimation_2d2d(kp1, kp2, matches):
    # 相机内参,TUM Freiburg2
    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    # 把匹配点转换为numpy数组
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # 计算基础矩阵
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)
    print("fundamental_matrix is \n", F)
    
    # 计算本质矩阵
    E = K.T @ F @ K
    print("essential_matrix is \n", E)
    
    # 计算单应矩阵
    H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC, 3)
    print("homography_matrix is \n", H)
    
    # 从本质矩阵中恢复旋转和平移信息
    _, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)
    print("R is \n", R)
    print("t is \n", t)
    
    return R, t

def pixel2cam(p, K):
    return np.array([(p[0] - K[0, 2]) / K[0, 0], (p[1] - K[1, 2]) / K[1, 1]])

def main(img1_path, img2_path):
    # 读取图像
    img1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
    img2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)
    
    kp1, kp2, matches = find_feature_matches(img1, img2)
    print(f"一共找到了 {len(matches)} 组匹配点")
    
    # 估计两张图像间运动
    R, t = pose_estimation_2d2d(kp1, kp2, matches)
    
    # 验证E=t^R*scale
# 验证E=t^R*scale
    t = t.ravel()  # 将t转换为一维数组
    t_x = np.array([[0, -t[2], t[1]],
                    [t[2], 0, -t[0]],
                    [-t[1], t[0], 0]])
    print("t^R=\n", t_x @ R)

    
    # 验证对极约束
    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    for m in matches:
        pt1 = pixel2cam(kp1[m.queryIdx].pt, K)
        y1 = np.array([pt1[0], pt1[1], 1])
        pt2 = pixel2cam(kp2[m.trainIdx].pt, K)
        y2 = np.array([pt2[0], pt2[1], 1])
        d = y2.T @ t_x @ R @ y1
        print(f"epipolar constraint = {d}")

if __name__ == "__main__":
    # if len(sys.argv) != 3:
    #     print("Usage: python pose_estimation_2d2d.py img1 img2")
    #     sys.exit(1)
    # main(sys.argv[1], sys.argv[2])
    main("./1.png", "./2.png")


一共找到了 79 组匹配点
fundamental_matrix is 
 [[ 4.54443750e-06  1.33385558e-04 -1.79849925e-02]
 [-1.27565701e-04  2.26679480e-05 -1.41667843e-02]
 [ 1.81499464e-02  4.14605587e-03  1.00000000e+00]]
essential_matrix is 
 [[  1.23307318  36.19935978   8.75048342]
 [-34.61991535   6.15301049 -26.03864593]
 [ -6.36842577  27.70151129   0.9175421 ]]
homography_matrix is 
 [[ 9.21362499e-01 -1.68936474e-01  3.86967804e+01]
 [ 3.72643369e-02  9.20111854e-01  1.29330721e+01]
 [-3.75793856e-05 -4.30481904e-05  1.00000000e+00]]
R is 
 [[ 0.99673436 -0.0782798   0.01982133]
 [ 0.07700415  0.99531145  0.05852755]
 [-0.02430992 -0.05681009  0.99808899]]
t is 
 [[-0.58108399]
 [-0.1672115 ]
 [ 0.79648083]]
t^R=
 [[-0.05726743 -0.78324719 -0.21350803]
 [ 0.7797537  -0.09535979  0.59576085]
 [ 0.12191957 -0.59144883 -0.03069506]]
epipolar constraint = -0.007110799685962909
epipolar constraint = -0.019245563704180985
epipolar constraint = 0.005926078826812814
epipolar constraint = -0.023511313327429437
epipo

三角测量，和pose_estimation_2d2d.cpp中的原理大致类似，只不过是通过三角型的关系来

In [3]:
import numpy as np
import cv2
import sys

def find_feature_matches(img1, img2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = matcher.match(des1, des2)
    
    min_dist = min(matches, key=lambda x: x.distance).distance
    good_matches = [m for m in matches if m.distance <= max(2 * min_dist, 30.0)]
    
    return kp1, kp2, good_matches

def pose_estimation_2d2d(kp1, kp2, matches):
    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    E, mask = cv2.findEssentialMat(pts1, pts2, K, cv2.RANSAC)
    
    _, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)
    
    return R, t

def triangulation(kp1, kp2, matches, R, t):
    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    T1 = np.eye(4)
    T2 = np.eye(4)
    T2[:3, :3] = R
    T2[:3, 3] = t.ravel()
    
    pts1 = np.float32([pixel2cam(kp1[m.queryIdx].pt, K) for m in matches]).T
    pts2 = np.float32([pixel2cam(kp2[m.trainIdx].pt, K) for m in matches]).T
    
    pts_4d = cv2.triangulatePoints(T1[:3], T2[:3], pts1, pts2)
    
    points = []
    for i in range(pts_4d.shape[1]):
        x = pts_4d[:, i]
        x /= x[3]
        points.append(x[:3])
    
    return points

def pixel2cam(p, K):
    return [(p[0] - K[0, 2]) / K[0, 0], (p[1] - K[1, 2]) / K[1, 1]]

def main(img1_path, img2_path):
    img1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
    img2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)
    
    kp1, kp2, matches = find_feature_matches(img1, img2)
    print(f"一共找到了 {len(matches)} 组匹配点")
    
    R, t = pose_estimation_2d2d(kp1, kp2, matches)
    
    points = triangulation(kp1, kp2, matches, R, t)
    
    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    for i, m in enumerate(matches):
        pt1_cam = pixel2cam(kp1[m.queryIdx].pt, K)
        pt1_cam_3d = [points[i][0] / points[i][2], points[i][1] / points[i][2]]
        
        print(f"point in the first camera frame: {pt1_cam}")
        print(f"point projected from 3D {pt1_cam_3d}, d={points[i][2]}")
        
        pt2_cam = pixel2cam(kp2[m.trainIdx].pt, K)
        pt2_trans = R @ np.array(points[i]) + t.ravel()
        pt2_trans /= pt2_trans[2]
        print(f"point in the second camera frame: {pt2_cam}")
        print(f"point reprojected from second frame: {pt2_trans[:2]}")
        print()

if __name__ == "__main__":
    # if len(sys.argv) != 3:
    #     print("Usage: python triangulation.py img1 img2")
    #     sys.exit(1)
    # main(sys.argv[1], sys.argv[2])
    main("./1.png", "./2.png")


一共找到了 79 组匹配点
point in the first camera frame: [np.float64(-0.1537723171434057), np.float64(-0.07428023032629556)]
point projected from 3D [np.float32(-0.15373677), np.float32(-0.07464735)], d=12.89017391204834
point in the second camera frame: [np.float64(-0.1794970126636291), np.float64(-0.05777350076100646)]
point reprojected from second frame: [-0.17954932 -0.05741991]

point in the first camera frame: [np.float64(-0.013630255327318147), np.float64(-0.3026871401151631)]
point projected from 3D [np.float32(-0.013613506), np.float32(-0.30351782)], d=21.794363021850586
point in the second camera frame: [np.float64(-0.004031483970051877), np.float64(-0.2700575815738963)]
point reprojected from second frame: [-0.0040815 -0.2692411]

point in the first camera frame: [np.float64(-0.4686120176617394), np.float64(0.1195777351247601)]
point projected from 3D [np.float32(-0.46861202), np.float32(0.11957773)], d=11.227673530578613
point in the second camera frame: [np.float64(-0.49932808600499

# 这个3D2D用pnp来搞

In [5]:
import numpy as np
import cv2
import sys
from scipy.optimize import least_squares

def find_feature_matches(img1, img2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = matcher.match(des1, des2)
    
    min_dist = min(matches, key=lambda x: x.distance).distance
    good_matches = [m for m in matches if m.distance <= max(2 * min_dist, 30.0)]
    
    return kp1, kp2, good_matches

def pixel2cam(p, K):
    return [(p[0] - K[0, 2]) / K[0, 0], (p[1] - K[1, 2]) / K[1, 1]]

def project(points, rvec, tvec, K):
    R, _ = cv2.Rodrigues(rvec)
    points_proj = np.dot(R, points.T) + tvec.reshape(3, 1)  # 修改这一行
    points_proj = points_proj.T
    points_proj = points_proj[:, :2] / points_proj[:, 2:][:, np.newaxis]  # 修改这一行
    points_proj = np.dot(points_proj, K.T[:2, :2]) + K[:2, 2]
    return points_proj


def objective_function(params, points_3d, points_2d, K):
    rvec = params[:3]
    tvec = params[3:]
    points_proj = project(points_3d, rvec, tvec, K)
    return (points_proj - points_2d).ravel()

def bundleAdjustment(points_3d, points_2d, K, rvec, tvec):
    params = np.hstack((rvec.ravel(), tvec.ravel()))
    result = least_squares(objective_function, params, args=(points_3d, points_2d, K))
    rvec_opt = result.x[:3].reshape(3, 1)  # 修改这一行
    tvec_opt = result.x[3:].reshape(3, 1)  # 修改这一行
    R_opt, _ = cv2.Rodrigues(rvec_opt)
    return R_opt, tvec_opt


def main(img1_path, img2_path, depth1_path):
    img_1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
    img_2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)
    depth1 = cv2.imread(depth1_path, cv2.IMREAD_UNCHANGED)

    kp1, kp2, matches = find_feature_matches(img_1, img_2)
    print(f"一共找到了 {len(matches)} 组匹配点")

    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    pts_3d = []
    pts_2d = []
    for m in matches:
        d = depth1[int(kp1[m.queryIdx].pt[1]), int(kp1[m.queryIdx].pt[0])]
        if d == 0:
            continue
        dd = d / 5000.0
        p1 = pixel2cam(kp1[m.queryIdx].pt, K)
        pts_3d.append([p1[0] * dd, p1[1] * dd, dd])
        pts_2d.append(kp2[m.trainIdx].pt)

    print(f"3d-2d pairs: {len(pts_3d)}")

    pts_3d = np.array(pts_3d)
    pts_2d = np.array(pts_2d)

    print("solving PnP in OpenCV...")
    _, rvec, tvec = cv2.solvePnP(pts_3d, pts_2d, K, None)
    rvec = rvec.reshape(3, 1)  # 添加这一行
    tvec = tvec.reshape(3, 1)  # 添加这一行
    print(f"R=\n", cv2.Rodrigues(rvec)[0])
    print(f"t=\n", tvec)


    print("calling bundle adjustment")
    R_est, t_est = bundleAdjustment(pts_3d, pts_2d, K, rvec, tvec)
    print("R_est=\n", R_est)
    print("t_est=\n", t_est)

if __name__ == "__main__":
    # if len(sys.argv) != 4:
    #     print("Usage: python pose_estimation_3d2d.py img1 img2 depth1")
    #     sys.exit(1)
    # main(sys.argv[1], sys.argv[2], sys.argv[3])
    main("./1.png", "./2.png", "./1_depth.png")




一共找到了 79 组匹配点
3d-2d pairs: 75
solving PnP in OpenCV...
R=
 [[ 0.99790591 -0.0509194   0.03988747]
 [ 0.04981866  0.99836232  0.02812094]
 [-0.04125405 -0.02607491  0.99880839]]
t=
 [[-0.12678213]
 [-0.00843949]
 [ 0.06034935]]
calling bundle adjustment
R_est=
 [[ 0.9968297  -0.04872288  0.06290182]
 [ 0.03817226  0.9865074   0.15920436]
 [-0.06981001 -0.15629853  0.98523973]]
t_est=
 [[-0.18861735]
 [-0.18932136]
 [ 0.15602084]]


# 这个3D3D用ICP来搞

In [6]:
import numpy as np
import cv2
import sys
from scipy.optimize import least_squares

def find_feature_matches(img1, img2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = matcher.match(des1, des2)
    
    min_dist = min(matches, key=lambda x: x.distance).distance
    good_matches = [m for m in matches if m.distance <= max(2 * min_dist, 30.0)]
    
    return kp1, kp2, good_matches

def pixel2cam(p, K):
    return [(p[0] - K[0, 2]) / K[0, 0], (p[1] - K[1, 2]) / K[1, 1]]

def pose_estimation_3d3d(pts1, pts2):
    p1 = np.mean(pts1, axis=0)
    p2 = np.mean(pts2, axis=0)
    q1 = pts1 - p1
    q2 = pts2 - p2

    W = np.dot(q1.T, q2)
    U, _, Vt = np.linalg.svd(W)
    R = np.dot(U, Vt)

    if np.linalg.det(R) < 0:
        U[:, 2] *= -1
        R = np.dot(U, Vt)

    t = p1 - np.dot(R, p2)

    return R, t

def objective_function(params, pts1, pts2):
    R = params[:9].reshape(3, 3)
    t = params[9:]
    transformed_pts2 = np.dot(R, pts2.T).T + t
    error = pts1 - transformed_pts2
    return error.ravel()

def bundle_adjustment(pts1, pts2, R, t):
    params = np.hstack((R.ravel(), t))
    result = least_squares(objective_function, params, args=(pts1, pts2))
    R_opt = result.x[:9].reshape(3, 3)
    t_opt = result.x[9:]
    return R_opt, t_opt

def main(img1_path, img2_path, depth1_path, depth2_path):
    img1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
    img2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)
    depth1 = cv2.imread(depth1_path, cv2.IMREAD_UNCHANGED)
    depth2 = cv2.imread(depth2_path, cv2.IMREAD_UNCHANGED)

    kp1, kp2, matches = find_feature_matches(img1, img2)
    print(f"一共找到了 {len(matches)} 组匹配点")

    K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
    
    pts1 = []
    pts2 = []
    for m in matches:
        d1 = depth1[int(kp1[m.queryIdx].pt[1]), int(kp1[m.queryIdx].pt[0])]
        d2 = depth2[int(kp2[m.trainIdx].pt[1]), int(kp2[m.trainIdx].pt[0])]
        if d1 == 0 or d2 == 0:
            continue
        p1 = pixel2cam(kp1[m.queryIdx].pt, K)
        p2 = pixel2cam(kp2[m.trainIdx].pt, K)
        pts1.append([p1[0] * d1 / 5000.0, p1[1] * d1 / 5000.0, d1 / 5000.0])
        pts2.append([p2[0] * d2 / 5000.0, p2[1] * d2 / 5000.0, d2 / 5000.0])

    pts1 = np.array(pts1)
    pts2 = np.array(pts2)
    print(f"3d-3d pairs: {pts1.shape[0]}")

    R, t = pose_estimation_3d3d(pts1, pts2)
    print("ICP via SVD results:")
    print(f"R =\n{R}")
    print(f"t =\n{t}")

    R_opt, t_opt = bundle_adjustment(pts1, pts2, R, t)
    print("Bundle Adjustment results:")
    print(f"R =\n{R_opt}")
    print(f"t =\n{t_opt}")

    # Verify p1 = R * p2 + t
    for i in range(5):
        print(f"p1 = {pts1[i]}")
        print(f"p2 = {pts2[i]}")
        print(f"(R * p2 + t) = {np.dot(R_opt, pts2[i]) + t_opt}")
        print()

if __name__ == "__main__":
    # if len(sys.argv) != 5:
    #     print("Usage: python pose_estimation_3d3d.py img1 img2 depth1 depth2")
    #     sys.exit(1)
    # main(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4])
    main("./1.png", "./2.png", "./1_depth.png", "./2_depth.png")


一共找到了 79 组匹配点
3d-3d pairs: 72
ICP via SVD results:
R =
[[ 0.99694524  0.05983348 -0.05020111]
 [-0.05932608  0.99817197  0.01153856]
 [ 0.05079973 -0.00852508  0.99867247]]
t =
[ 0.14415983 -0.06667848 -0.03009795]
Bundle Adjustment results:
R =
[[ 1.00087517  0.08272947 -0.02778242]
 [-0.06703671  1.31614924  0.15213556]
 [ 0.10589317 -1.00626747  0.40297655]]
t =
[ 0.10824064 -0.28510033  0.92021366]
p1 = [-0.24369837 -0.11771931  1.5848    ]
p2 = [-0.29721115 -0.09566136  1.6558    ]
(R * p2 + t) = [-0.24314677 -0.13917484  1.65225051]

p1 = [-0.03741232 -0.83081566  2.7448    ]
p2 = [-0.01114786 -0.74676322  2.7652    ]
(R * p2 + t) = [-0.04152024 -0.84651961  2.78478747]

p1 = [-0.62775266  0.16018633  1.3396    ]
p2 = [-0.70964508  0.15903255  1.4212    ]
(R * p2 + t) = [-0.62835319  0.18799757  1.25774808]

p1 = [ 0.40204515 -0.34182104  2.2068    ]
p2 = [ 0.3788106  -0.26285858  2.2196    ]
(R * p2 + t) = [ 0.40397075 -0.31877558  2.1192799 ]

p1 = [ 0.6271664  -0.01663175  1.5